# Geodata Worked example

1. We're going to write the geoload.py code (this is a restartable process)
2. The code is going to read a file that is the list of the locations we ultimately want to have
3. The file with the locations is called where.data
4. Geoload is going to read all the locations we want and then and then starting ti pull them down and then it's going to simply insert the JSON into this geodata.sqlite file
5. We can visualize the contents of the database with the code geodump.py
6. geodump.py reads the database and produces a file called where.js and writes a debug output
7. Then, the where.js file is read into when you open a browser
8. The file where.html opens in a browser and it reads the where.js and then it visualizes it

First, we will go to https://www.py4e.com/code3/opengeo.zip and download the folder.

That folder has a README.txt, the geodump.py and geoload.py files, and the where.data, where.html, and where.js files.

- The where.html and where.js are visualization part of it
- geoload.py is the spidering part of it
- geodump.py is the read the database past of it

## Geoload

First, we will look at geoload.py, which is the restartable spider and it's going to be looking at a proxy of the GeoAPI

In [4]:
# geoload.py
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

# https://py4e-data.dr-chuck.net/opengeo?q=Ann+Arbor%2C+MI
serviceurl = 'https://py4e-data.dr-chuck.net/opengeo?'

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

#Create or open an existing database
conn = sqlite3.connect('opengeo.sqlite')
cur = conn.cursor()

#Create a simple table. We're going to take the address that we looked up as the key 
# and the JSON that we get back as the value
cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# First, we're going to open the where.data file and then read the whole file
fh = open("where.data")
count = 0
nofound = 0
for line in fh:
    # If we get past 100 retrieved locations, then we're going to stop 
    # to let our API settle down
    if count > 100 :
        print('Retrieved 100 locations, restart to retrieve more')
        break
    
    # Then we're going to take the address from the line, which is just a text line,
    # and then we're going to check to see if the address has already been retrieved
    address = line.strip()
    print('')
    #We're doing an encode and a memory view because of UTF-8 and because the database
    # is outside of us
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        # Then, we're going to retrieve that record
        data = cur.fetchone()[0]
        # If the address is already in the database, then we're just going to continue
        # We're not going to retrieve the addresses that are already retrieved
        print("Found in database", address)
        continue
    except:
        pass

    parms = dict()
    parms['q'] = address

    #If the addresses aren't there, then we're going to create a URL using the dictionary format
    # above, but that needs to be url encoded and that's what the urllib.parse.urlencode does
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    # Then we open the URL and we ignore the SSL errors
    uh = urllib.request.urlopen(url, context=ctx)
    # Then we read it and decode the file (in this case in UTF-8) into the internal format used
    # by Python (Unicode)
    data = uh.read().decode()
    #Debug statement
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    #A count so that each run only does no more than 100
    count = count + 1

    # Then we parse it
    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    # Then we have a couple sanity checks to make sure I got real data
    if not js or 'features' not in js:
        print('==== Download error ===')
        print(data)
        break

    if len(js['features']) == 0:
        print('==== Object not found ====')
        nofound = nofound + 1

    # Insert the data obtained into the database
    cur.execute('''INSERT INTO Locations (address, geodata)
        VALUES ( ?, ? )''',
        (memoryview(address.encode()), memoryview(data.encode()) ) )

    # Then we commit it
    conn.commit()

    #Then we sleep 5 seconds every round that goes around and around and around
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

# At the end of the loop it prints the number of features and sends out a little message
if nofound > 0:
    print('Number of features for which the location could not be found:', nofound)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")




Found in database AGH University of Science and Technology

Found in database Academy of Fine Arts Warsaw Poland

Found in database American University in Cairo

Found in database Arizona State University

Found in database Athens Information Technology

Found in database BITS Pilani

Found in database Babcock University

Found in database Banaras Hindu University

Found in database Bangalore University

Found in database Baylor University

Found in database Beijing normal university

Found in database Belarusian State University

Found in database Belgrade University

Found in database Beloit College

Found in database Belorussian State University

Found in database Ben Gurion University

Found in database Boston University

Found in database California Polytechnic State University of San Luis Obispo

Found in database California State University San Bernardino

Found in database City of Westminster College

Found in database Columbia University

Found in database Cranfield Universit

## Where.data

Next, let's look at this where.data file
The idea at some point is that we want to know the locations so this is justthe input to R and this is just all the places, and we can add to this Jerk Pit, Ann Arbor, MI.

This is the to-do list, in a sense

The Python code goes and reads the file where.data and it retrieves the data, 10 at a time. If it stops, you can just restart the process and it will pick right where it left off.

The code was reading all the information in where.data and it was checking geodata.sqlite and it was already there, so there was no need to go to the API, and that's why that's a restartable process.

It's a restartable process because you don't need to keep going back to the API because you're only trying to retrieve data back from a database.

Geoload worries about the fact that the API may blow up on you, or it may be slow, or you might run out of time.


# Geodump
This tool, geodump.py it opens a database and reads all this stuff and starts writing the where.js file

In [5]:
import sqlite3
import json
import codecs

#This opens a database
conn = sqlite3.connect('opengeo.sqlite')
cur = conn.cursor()

#And then reads all of this stuff and starts writing the where.js file
cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    #It reads the first row and parses the JSON
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    #Checks to see if the features is there, which is an array of features
    if len(js['features']) == 0: continue

    try:
        #features, sub zero, because features is an array, sub geometry, sub coordinates, sub one. This is to get the latitude
        lat = js['features'][0]['geometry']['coordinates'][1]
        #features, sub zero, because features is an array, sub geometry, sub coordinates, sub one. This is to get the longitude
        lng = js['features'][0]['geometry']['coordinates'][0]
        where = js['features'][0]['properties']['display_name']
        where = where.replace("'", "")
    except:
        print('Unexpected format')
        print(js)

    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")

AGH University of Science and Technology 50.065703299999996 19.918958667058632
Academy of Fine Arts in Warsaw 52.2394966 21.015693129786936
American University in Cairo 30.04281175 31.23658590813949
Arizona State University 33.4213174 -111.93316305413154
Athens Information Technology 37.9425172 23.8706149
BITS Pilani 17.547185 78.572791
Babcock University 6.8900558499999995 3.722437696387937
Banaras Hindu University 25.2663747 82.99046480176699
Bangalore University 12.94393425 77.5091805440154
Baylor University 31.55043405 -97.1102905567766
Beijing Normal University (BNU) 39.959732 116.35973697755406
Belarusian State University 53.83806745 27.476155257195504
Belgrade University 44.81842 20.45757
Beloit College 42.5009819 -89.0157777411969
Unexpected format
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-90.45997, 35.95422]}, 'properties': {'country_code': 'us', 'country': 'United States', 'county': 'Craighead County', 'datas

When you run this it's really fast because it's just reading from a database and writing to where.js.

If we look at where.js, it's just a JavaScript variable that is all of this data that is all of this data with longitude, latitutde, and the name, and that's how you put pins on a map

# where.js

In [ ]:
myData = [
    [50.065703299999996,19.918958667058632, 'AGH University of Science and Technology'],
    [52.2394966,21.015693129786936, 'Academy of Fine Arts in Warsaw'],
    [30.04281175,31.23658590813949, 'American University in Cairo'],
    [33.4213174,-111.93316305413154, 'Arizona State University'],
    [37.9425172,23.8706149, 'Athens Information Technology'],
    [17.547185,78.572791, 'BITS Pilani'],
    [6.8900558499999995,3.722437696387937, 'Babcock University'],
    [25.2663747,82.99046480176699, 'Banaras Hindu University'],
    [12.94393425,77.5091805440154, 'Bangalore University'],
    [31.55043405,-97.1102905567766, 'Baylor University'],
    [39.959732,116.35973697755406, 'Beijing Normal University (BNU)'],
    [53.83806745,27.476155257195504, 'Belarusian State University'],
    [44.81842,20.45757, 'Belgrade University'],
    [42.5009819,-89.0157777411969, 'Beloit College'],
    [35.95422,-90.45997, 'Beloit College'],
    [31.2631254,34.802238544584185, 'Ben Gurion University of the Negev'],
    [42.350421499999996,-71.10322831831216, 'Boston University'],
    [35.30059,-120.66004, 'California Polytechnic State University - San Luis Obispo'],
    [34.18191745,-117.32174391836504, 'California State University, San Bernardino'],
    [51.5213399,-0.1748934415893599, 'City of Westminster College'],
    [40.80775585,-73.96161411567529, 'Columbia University'],
    [52.070132900000004,-0.6287023427161589, 'Cranfield University'],
    [50.1029026,14.3946353, 'Czech Technical University in Prague'],
    [37.31770065,-122.04455706238198, 'De Anza College'],
    [48.434864649999994,35.03778389976746, 'Dnipro National University'],
    [30.2851494,-97.73393515146053, 'University of Texas at Austin'],
    [36.00015569999999,-78.94422972195878, 'Duke University'],
    [45.732398,4.835571, '7th Arr.'],
    [48.8359694,2.3214992, 'École centrale dhypnose'],
    [36.1063398,-79.50273682932499, 'Elon University'],
    [55.48841555,8.447026487581917, 'Erhvervsakademi Sydvest'],
    [-2.147712,-79.9647289, 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL'],
    [51.24839645,6.770322755895527, 'Fachhochschule Düsseldorf'],
    [47.72360745,13.086718915696402, 'Fachhochschule Salzburg'],
    [45.2461821,19.851402622733332, 'Faculty of Technical Sciences'],
    [40.75278,-73.42778, 'Farmingdale State University of New York'],
    [26.119403650000002,-80.14169191096471, 'Florida Atlantic University'],
    [43.2125814,-71.4947929, 'Franklin Pierce College'],
    [26.153410700000002,91.6657647108439, 'Gauhati University'],
    [38.83133325,-77.30798838879116, 'George Mason University'],
    [38.897836749999996,-77.01292415460526, 'Georgetown University Law Center'],
    [33.7535889,-84.38542086620515, 'Georgia State University'],
    [50.87485475,4.707931057125883, 'Campus Groep T'],
    [21.0055269,105.84359264177206, 'Hanoi University of Science and Technology'],
    [31.77293975,35.197994954152726, 'Hebrew University - Givat Ram Campus'],
    [17.445824,78.349309, 'IIIT-H Library'],
    [26.5093058,80.24823225761622, 'IIT Kanpur'],
    [45.4378506,12.3226823, 'Bar Iuav'],
    [41.83616405,-87.62647227745128, 'Illinois Institute of Technology'],
    [40.51799375,-89.00846070697986, 'Illinois State University'],
    [25.2621544,82.9925896, 'Indian Institute of Technology (BHU)'],
    [-42.118837799999994,171.8690191894505, 'School of Mines'],
    [39.770318,-86.177058, 'Indiana University'],
    [43.879022,-85.800815, 'Indiana University'],
    [14.7168914,-17.4556123, 'Institut Supérieur de Technologies'],
    [18.4876505,-69.96210823269232, 'Instituto Tecnológico de Santo Domingo'],
    [17.4454957,78.34854697544472, 'International Institute of Information Technology, Hyderabad'],
    [35.95422,-90.45997, 'International Institute of Information Technology, Hyderabad'],
    [22.49952155,88.37114925302436, 'Jadavpur University'],
    [16.9786772,82.2411916, 'Jawaharlal Nehru Technological University (JNTUK)'],
    [28.540166749999997,77.16456005394733, 'Jawaharlal Nehru University'],
    [32.49339995,35.989854503370836, 'Jordan University of Science and Technology'],
    [39.18825185,-96.58258555991623, 'K-State Student Union'],
    [42.29075735,-85.59936465969835, 'Kalamazoo College'],
    [54.898805249999995,23.91265256983673, 'Kaunas University of Technology'],
    [54.898805249999995,23.91265256983673, 'Kaunas University of Technology'],
    [55.79037155,49.12070180283408, 'Kazan Federal University'],
    [41.144232450000004,-81.33983207322399, 'Kent State University'],
    [13.65167,100.49556, 'King Mongkuts University of Technology'],
    [35.902915,-84.678361, 'King Mongkuts University of Technology'],
    [50.4507932,30.466127, 'Politekhnichnyi Instytut'],
    [50.44892,30.45891, 'Kyiv Polytechnical Institute'],
    [50.44209,30.50821, 'Taras Shevchenko National University of Kyiv'],
    [46.4666416,-80.9738711, 'Laurentian University'],
    [10.4730027,-66.8928687, 'Lisandro Alvarado'],
    [18.0164751,-76.743473, 'University of Technology'],
    [49.362304,23.320647, 'University of Technology'],
    [42.70237935,-84.48038686520441, 'Michigan State University'],
    [13.06686,80.28347, 'Madras University Library'],
    [39.416095049999996,-81.4484225136152, 'Marietta College'],
    [24.4348507,54.6167989, 'Masdar Institute'],
    [44.8036358,20.4952256, 'Matematički fakultet'],
    [39.875496749999996,32.78553505584672, 'Middle East Technical University'],
    [37.9532435,-91.77426666814159, 'Missouri University of Science and Technology'],
    [-37.8774234,145.04496691740303, 'Monash University (Caulfield campus)'],
    [-37.910776350000006,145.13480648790716, 'Monash University, Clayton Campus'],
    [34.527022,-92.987376, 'Monash University, Clayton Campus'],
    [22.135603,113.545535, 'Institute of Applied Physics and Materials Engineering'],
    [55.60460415,38.10651819116069, 'Moscow Institute of Physics and Technology'],
    [55.70229715,37.53179772747929, 'Moscow State University'],
    [22.2501589,84.90668556980866, 'NIT Rourkela'],
    [36.0395283,138.3703646, 'Mt. Nyu'],
    [21.147717450000002,79.03924327117383, 'Rashtrasant Tukadoji Maharaj Nagpur University'],
    [1.3484104000000001,103.68293320728537, 'Nanyang Technological University'],
    [31.39571075,75.5357675438753, 'Dr. B. R. Ambedkar National Institute of Technology, Jalandhar'],
    [25.016817500000002,121.53845709079619, 'National Taiwan University'],
    [-12.02037945,-77.04816939032196, 'National University of Engineering'],
    [41.774365450000005,-88.14274924248255, 'North Central College'],
    [42.33895455,-71.08805803336392, 'Northeastern University'],
    [42.0557157,-87.67529454739416, 'Northwestern University'],
    [64.52944,40.54481, 'Arkhangelsk state technical university ("АГТУ") - 2th building'],
    [36.8862699,-76.30972478839735, 'Old Dominion University'],
    [42.2586441,-121.7854301598997, 'Oregon Institute of Technology'],
    [32.6210159,51.3919877470881, 'Payame Noor University'],
    [40.8025835,-77.85593833184565, 'Penn State University'],
    [45.0481479,9.7017191, 'Politecnico di Milano'],
    [44.439367950000005,26.050603266436525, 'Politehnica University of Bucharest'],
    [14.869264,120.998953, 'Politehnica University of Bucharest'],
    [12.02286805,79.85094146038637, 'Pondicherry University'],
    [-33.44177485,-70.64032584797746, 'Pontificia Universidad Católica de Chile'],
    [45.51181205,-122.68493059820187, 'Portland State University'],
    [40.430028,-86.92642114650494, 'Purdue University'],
    [12.9232191,77.50064642949692, 'R. V. College of Engineering'],
    [41.08224455,-74.1738235180645, 'Ramapo College of New Jersey'],
    [43.0826701,-77.67293215211362, 'Rochester Institute of Technology'],
    [10.7277931,79.0191745104736, 'Sastra Deemed University'],
    [59.9406782,30.296499938428813, 'St. Petersburg State University'],
    [59.857196849999994,30.328205524473546, 'Saint Petersburg State University of Aerospace Instrumentation'],
    [40.047389,-105.246478, 'Saint Petersburg State University of Aerospace Instrumentation'],
    [37.7245167,-122.47999485908868, 'San Francisco State University'],
    [37.33519025,-121.88122545292521, 'San José State University Main Campus'],
    [31.200891149999997,121.4283949471511, 'Shanghai Jiao Tong University (Xuhui Campus)'],
    [35.7025215,51.35172929975248, 'Sharif University of Technology'],
    [-19.0413367,-65.2631883, 'Simón Bolívar'],
    [49.276709600000004,-122.91780296438841, 'Simon Fraser University'],
    [54.7847989,32.04552520919078, 'Smolensk State University'],
    [38.33982655,-122.6734191695437, 'Sonoma State University'],
    [7.792138,6.731998, 'Federal University Lokoja'],
    [46.773410999999996,23.5985997891404, 'Spiru Haret University'],
    [37.431313849999995,-122.16936535498309, 'Stanford University'],
    [-22.8164,-47.06879, 'State University of Campinas (UNICAMP)'],
    [45.400544,-122.653682, 'State University of Campinas (UNICAMP)'],
    [-33.9326212,18.865055115199628, 'Stellenbosch University'],
    [42.05997765,-71.07932570575417, 'Stonehill College'],
    [59.4393125,24.772347058941484, 'Tallinn University'],
    [18.0164751,-76.743473, 'University of Technology'],
    [18.0164751,-76.743473, 'University of Technology'],
    [30.800208750000003,30.992227584817762, 'Tanta University'],
    [32.97973255,-97.3129042119206, 'Tarrant County College'],
    [35.6870811,-108.1473593, 'Navajo Technical University'],
    [32.774576,35.0236399, 'Technion'],
    [32.112382249999996,34.80501795095419, 'Tel Aviv University'],
    [42.605625,-77.136084, 'The Pinnacle'],
    [32.316307800000004,-95.25369943794593, 'The University of Texas at Tyler'],
    [51.57236815,-1.3114536688023473, 'The University of Manchester at Harwell'],
    [-26.1888766,28.024791203071317, 'University of the Witwatersrand'],
    [50.871738,4.331478, 'Transilvania'],
    [42.40649125,-71.11800725402092, 'Tufts University'],
    [37.8713958,-122.2651394, 'UC Berkeley'],
    [-34.508711500000004,-58.68837906432547, 'Universidad de Buenos Aires'],
    [43.080274450000005,-89.43095871991434, 'University of Wisconsin-Madison'],
    [10.4901789,-66.89166332272254, 'Universidad Central de Venezuela'],
    [40.4314742,-3.7900439484094033, 'Universidad Complutense de Madrid (Campus de Somosaguas)'],
    [4.62999775,-74.06878622678896, 'Universidad Cooperativa de Colombia'],
    [19.32160295,-99.18493018260631, 'National Autonomous University of Mexico'],
    [10.0003416,-84.10616817673456, 'Universidad Nacional'],
    [4.63874255,-74.08523751358481, 'National University of Colombia'],
    [-16.50065545,-68.13434982328437, 'Universidad Tecnológica Boliviana'],
    [-34.508711500000004,-58.68837906432547, 'Universidad de Buenos Aires'],
    [38.9969805,-3.917926657779383, 'Universidad de Castilla-La Mancha'],
    [4.60168,-74.065229, 'Universidad de los Andes'],
    [8.338148499999999,-62.670820613258186, 'Universidad de Oriente'],
    [14.58348855,-90.55502036799915, 'Universidad de San Carlos de Guatemala'],
    [41.6516894,-4.7209841, 'Universidad de Valladolid'],
    [4.8609881,-74.03323666587279, 'Universidad de la Sabana'],
    [14.60511395,-90.4895231383772, 'Universidad del Valle de Guatemala'],
    [-6.7505593,-35.64545021761968, 'Universidade Federal da Paraíba/CAVN'],
    [-27.60341335,-48.5224424733554, 'Federal University of Santa Catarina'],
    [-30.05919315,-51.12802449315921, 'Propriedade da Universidade Federal do Rio Grande do Sul'],
    [-22.8533498,-43.233170287422354, 'Universidade Federal do Rio de Janeiro'],
    [38.69998655,-9.184641177083622, 'Palácio Burnay'],
    [-23.565220099999998,-46.71258298223419, 'Universidade de São Paulo (Portaria 1)'],
    [41.5573185,-8.3978382, 'Universidade do Minho'],
    [-7.7761108,110.3758574, 'Universitas Gadjah Mada'],
    [39.4808517,-0.3420311427062354, 'Campus de Vera de la Universitat Politècnica de València'],
    [50.40801,4.44833, 'Université Catholique De Louvain'],
    [53.3068763,-6.224625093340977, 'University College Dublin (Belfield Campus)'],
    [48.14907275,11.567444920339295, 'Technical University Munich'],
    [41.07573825,-81.51145320375983, 'University of Akron'],
    [53.52682,-113.5244937350756, 'University of Alberta'],
    [52.36813335,4.889804155733945, 'University of Amsterdam'],
    [36.0970389,-94.17033216657404, 'University of Arkansas'],
    [37.98785015,23.731825812552522, 'National Technical University of Athens'],
    [44.7912511,20.4784348, 'Faculty of Security Studies, University of Belgrade'],
    [52.44942455,-1.9305742840260405, 'University of Birmingham Edgbaston Campus'],
    [-37.3285798,-59.1385728, 'National University of Central Buenos Aires'],
    [52.210945550000005,0.09200497637871279, 'University of Cambridge'],
    [35.655730250000005,-97.47032859449375, 'University of Central Oklahoma'],
    [41.79139685,-87.60084387193544, 'The University of Chicago'],
    [39.1318613,-84.51576195582436, 'University of Cincinnati West Campus'],
    [40.00442,-105.26706, 'University of Colorado at Boulder'],
    [41.8111114,-72.2484133545308, 'University of Connecticut'],
    [32.8487635,-96.92256763370509, 'University of Dallas'],
    [47.5543249,21.6239011, 'Internation Education Office University of Debrecen'],
    [39.6803338,-75.7530748911031, 'University of Delaware'],
    [-34.962507,-71.216971, 'University of Delaware'],
    [51.8773639,0.9498750625114011, 'University of Essex'],
    [38.035611,-78.521408, 'University of Essex'],
    [29.6411884,-82.35622955129917, 'University of Florida'],
    [57.69854645,11.971242155543006, 'University of Gothenburg'],
    [54.08943465,13.404874267720633, 'University of Greifswald'],
    [35.191523,-97.442651, 'University of Greifswald'],
    [21.3320662,-157.9218882, 'University of Hawaii'],
    [60.22672205,25.017670049469757, 'University of Helsinki'],
    [8.484273949999999,4.673080587006053, 'University of Ilorin'],
    [9.6844884,80.02269268168564, 'University Of Jaffna'],
    [38.95350515,-95.26309220025998, 'University of Kansas'],
    [12.39316345,75.0923478747471, 'Central University Of Kerala'],
    [48.86017615,2.3151450411569674, 'University of London'],
    [36.7156622,-4.47168005070005, 'Botanical Garden of the University of Malaga'],
    [3.1257602,101.65587961386524, 'University of Malaya'],
    [53.466022550000005,-2.2330858190427176, 'University of Manchester'],
    [42.2942142,-83.71003893509601, 'University of Michigan'],
    [38.93641465,-92.32970301784542, 'University of Missouri'],
    [6.79722735,79.90122421406977, 'University of Moratuwa'],
    [19.07112,72.85629, 'University of Mumbai'],
    [40.8205941,-96.70260581368638, 'University of Nebraska-Lincoln City Campus'],
    [40.81752,-96.70057, 'University of Nebraska - Lincoln'],
    [41.29091365,-72.96263883692575, 'University of New Haven'],
    [-33.91759945,151.23126487290787, 'University of New South Wales'],
    [41.704567749999995,-86.23822026017265, 'University of Notre Dame du Lac'],
    [35.1959878,-97.44570827599858, 'University of Oklahoma'],
    [45.42252705,-75.68339041611182, 'University of Ottawa'],
    [51.750690000000006,-1.247481719363849, 'University of Oxford Botanic Garden'],
    [45.41667,11.86667, 'University of Padua'],
    [45.186926549999995,9.156876823159685, 'University of Pavia'],
    [39.950394450000005,-75.19467130666834, 'University of Pennsylvania'],
    [37.9417,23.653, 'University of Piraeus'],
    [-25.75433545,28.230857806174797, 'University of Pretoria'],
    [40.96214545,-5.66905879509083, 'University of Salamanca International Courses'],
    [-23.56096405,-46.72770807289966, 'University of São Paulo'],
    [43.82157735,18.309199627076516, 'International University in Sarajevo'],
    [34.02186895,-118.285857923125, 'University of Southern California'],
    [-33.8842684,18.62729953211983, 'University of Stellenbosch Business School'],
    [58.38101399999999,26.719559831081384, 'University of Tartu'],
    [35.70458395,51.39476104247319, 'University of Tehran'],
    [30.2851494,-97.73393515146053, 'University of Texas at Austin'],
    [43.663461999999996,-79.39775966248058, 'University of Toronto'],
    [48.53405945,9.071244429234808, 'Climate Garden of University Tuebingen'],
    [52.2476588,6.853631, 'University of Twente'],
    [40.762813699999995,-111.83687191368261, 'University of Utah'],
    [48.2131284,16.360685994114505, 'University of Vienna'],
    [52.2216394,21.00737979000361, 'Warsaw University of Technology'],
    [47.6554303,-122.30016924820552, 'University of Washington'],
    [47.7603126,-122.19153052756513, 'University of Washington Bothell'],
    [43.4701994,-80.54524293779451, 'University of Waterloo'],
    [30.5479646,-87.21620152987073, 'University of West Florida'],
    [43.0464346,-89.4301097018523, 'University of Wisconsin Arboretum'],
    [31.4932564,74.29502816771938, 'University of the Punjab'],
    [-26.1888766,28.024791203071317, 'University of the Witwatersrand'],
    [54.7219118,25.336616247800787, 'Vilnius Gediminas Technical University'],
    [54.682847,25.28784063939981, 'Vilnius University'],
    [37.546124649999996,-77.45406089907547, 'Virginia Commonwealth University'],
    [37.22192675,-80.42728184013652, 'Virginia Tech'],
    [18.519123,73.840719, 'Virginia Tech'],
    [52.2216394,21.00737979000361, 'Warsaw University of Technology'],
    [46.7337716,-117.14980348311619, 'Washington State University'],
    [42.4709046,-77.1055262, 'Washington State University'],
    [47.0844286,-118.9400452, 'Washington State University'],
    [31.907849900000002,34.81334092444211, 'Weizmann Institute of Science'],
    [15.024776,103.12992, 'Western University'],
    [39.1513078,-84.47083471474068, 'Xavier University'],
    [30.588607449999998,31.48192973657335, 'Zagazig University'],
    [33.6842062,73.0555742, 'Allama Iqbal Open University'],
    [33.4213174,-111.93316305413154, 'Arizona State University'],
    [35.627454,-99.385053, 'Arizona State University'],
    [39.20883,-96.58845116318602, 'Kansas State University'],
    [40.4314742,-3.7900439484094033, 'Universidad Complutense de Madrid (Campus de Somosaguas)'],
    [38.2187045,21.747894895431557, 'University of Patras'],
    [45.41667,11.86667, 'University of Padua'],
    [42.2790854,-83.7397637, 'Jerk Pit, Ann Arbor, MI']
    ];
    

If you watch where.html, it gives you some HTML to use OpenGeo to visualize the data

# where.html

In [ ]:
<!doctype html>
<html lang="pl">
  <head>
    <meta charset="utf-8">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/openlayers/openlayers.github.io@master/en/v6.4.3/css/ol.css" type="text/css">
    <style>
	  body {font-family: Verdana, Geneva, sans-serif;}
      .map {
        height: 400px;
        width: 100%;
      }
	  .ol-popup {
        position: absolute;
        background-color: white;
        -webkit-filter: drop-shadow(0 1px 4px rgba(0,0,0,0.2));
        filter: drop-shadow(0 1px 4px rgba(0,0,0,0.2));
        padding: 15px;
        border-radius: 10px;
        border: 1px solid #cccccc;
        bottom: 12px;
        left: -50px;
        min-width: 150px;
      }
      .ol-popup:after, .ol-popup:before {
        top: 100%;
        border: solid transparent;
        content: " ";
        height: 0;
        width: 0;
        position: absolute;
        pointer-events: none;
      }
      .ol-popup:after {
        border-top-color: white;
        border-width: 10px;
        left: 48px;
        margin-left: -10px;
      }
      .ol-popup:before {
        border-top-color: #cccccc;
        border-width: 11px;
        left: 48px;
        margin-left: -11px;
      }
      .ol-popup-closer {
        text-decoration: none;
        position: absolute;
        top: 2px;
        right: 8px;
      }
      .ol-popup-closer:after {
        content: "x";
      }
    </style>
    <script src="https://cdn.jsdelivr.net/gh/openlayers/openlayers.github.io@master/en/v6.4.3/build/ol.js"></script>
	<script src="where.js"></script>
    <title>Mapa informacji</title>
  </head>
  <body>
    <div id="map" class="map"></div>
	<div id="popup" class="ol-popup">
      <a href="#" id="popup-closer" class="ol-popup-closer"></a>
      <div id="popup-content"></div>
    </div>
	<p><b>About the map</b></p>
    <p>If you would like to make this map yourself, please visit <a href="https://www.py4e.com/">www.py4e.com</a></p>
	<p>The original Polish version of the code for this map came from <a href="https://py4e.pl">py4e.pl</a>.
    <p>Map generated <script>document.write(new Date());</script></p>
	</p>
    <script type="text/javascript">
      /**
       * Elements that make up the popup.
       */
      var container = document.getElementById('popup');
      var content = document.getElementById('popup-content');
      var closer = document.getElementById('popup-closer');


      /**
       * Add a click handler to hide the popup.
       * @return {boolean} Don't follow the href.
       */
      closer.onclick = function() {
        overlay.setPosition(undefined);
        closer.blur();
        return false;
      };


      /**
       * Create an overlay to anchor the popup to the map.
       */
      var overlay = new ol.Overlay({
        element: container,
        autoPan: true,
        autoPanAnimation: {
          duration: 250
        }
      });
	
	
      var map = new ol.Map({
        target: 'map',
		overlays: [overlay],
        layers: [
          new ol.layer.Tile({
            source: new ol.source.OSM()
          })
        ],
        view: new ol.View({
          center: ol.proj.fromLonLat([16.119735, 48.098628]),
          zoom: 4
        })
      });
	  
	var features = [];


	for ( pos in myData ) {
		var row = myData[pos];
		var longitude = row[1];
		var latitude = row[0];

		var iconFeature = new ol.Feature({
			geometry: new ol.geom.Point(ol.proj.transform([longitude, latitude], 'EPSG:4326', 'EPSG:3857')),
			name: row[2]
		});

		var iconStyle = new ol.style.Style({
			image: new ol.style.Icon(({
				anchor: [0.5, 1],
				src: "https://cdn.mapmarker.io/api/v1/font-awesome/v5/pin?icon=fa-university&size=40&hoffset=0&voffset=0"
			}))
		});

		iconFeature.setStyle(iconStyle);
		features.push(iconFeature);

	}

	var vectorSource = new ol.source.Vector({
		features: features
	});

	var vectorLayer = new ol.layer.Vector({
		source: vectorSource
	});
	map.addLayer(vectorLayer);
	
	/**
       * Add a click handler to the map to render the popup.
       */
      map.on('singleclick', function(evt) {
        var name = map.forEachFeatureAtPixel(evt.pixel, function(feature) {
          return feature.get('name');
        })
        if (name) {
          container.style.display="block";
          var coordinate = evt.coordinate;
          content.innerHTML = name;
          overlay.setPosition(coordinate);
        } else {
          container.style.display="none";
        }
      });
      map.on('pointermove', function(evt) {
        map.getTargetElement().style.cursor = map.hasFeatureAtPixel(evt.pixel) ? 'pointer' : '';
      });
	 
	/* http://www.geocodezip.com/OL_5.3.0_simpleMultipleMarkerExample.html */
	/* https://openlayers.org/ */
    </script>
  </body>
</html>

This creates a map that shows you all of the retrieved locations.